# EstrelaBet - Feature Engineering

## Objective
Create a comprehensive feature set for predicting **customer churn** (customers who do NOT make a redeposit after their first bet).

## Target Variable
- **Churn = 1**: Customer did NOT make a redeposit (churned)
- **Churn = 0**: Customer made a redeposit (retained)

## Feature Categories
1. **First Session Features** - Early indicators from initial user interaction
2. **Behavioral Features** - Aggregated behavioral patterns
3. **Financial Features** - Betting and monetary patterns
4. **Engagement Features** - Platform engagement metrics
5. **Trend Features** - Behavioral change indicators

---

## 1. Setup and Data Loading

In [1]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')

In [3]:
# Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from scipy import stats

pd.set_option('display.max_columns', 50)

In [4]:
# Load the raw dataset
df = pd.read_csv('../data/test_dataset.csv')

print(f"Dataset Shape: {df.shape}")
print(f"Unique Users: {df['user_id'].nunique()}")

Dataset Shape: (10000, 29)
Unique Users: 1446


In [5]:
# Convert timestamps and sort
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['user_id', 'timestamp']).reset_index(drop=True)

---
## 2. Target Variable Construction

In [ ]:
def create_target_variable(df):
    """
    Create target: Did user churn (NOT make a redeposit) after their first session?
    
    Target:
        - 1 = Churned (no redeposit after first session)
        - 0 = Retained (made a redeposit after first session)
    """
    df_sorted = df.sort_values(['user_id', 'timestamp']).copy()
    
    # Mark first session
    df_sorted['is_first_session'] = ~df_sorted.duplicated('user_id', keep='first')
    
    # Check for churn (no redeposit) after first session
    def check_churn(group):
        subsequent = group[~group['is_first_session']]
        if len(subsequent) == 0:
            return 1  # No subsequent sessions = churned
        return 0 if (subsequent['deposit_amount'].fillna(0) > 0).any() else 1
    
    target = df_sorted.groupby('user_id').apply(check_churn)
    target.name = 'churn'
    
    return target, df_sorted

In [ ]:
# Create target variable
target, df_processed = create_target_variable(df)

print(f"Target Distribution:")
print(target.value_counts())
print(f"\nChurn Rate: {target.mean():.2%}")
print(f"Retention Rate: {1 - target.mean():.2%}")

---
## 3. First Session Features

Extract features from the user's first session - early indicators of user behavior.

In [8]:
# Get first session for each user
first_sessions = df_processed[df_processed['is_first_session']].copy()

In [9]:
# Initialize first session features
first_session_features = pd.DataFrame()
first_session_features['user_id'] = first_sessions['user_id']

In [10]:
# Temporal features
first_session_features['first_session_hour'] = first_sessions['hour'].values
first_session_features['first_session_day_of_week'] = first_sessions['day_of_week'].values
first_session_features['first_session_weekend'] = first_sessions['is_weekend'].values
first_session_features['first_session_holiday'] = first_sessions['is_holiday'].values

In [11]:
# Game and device features
first_session_features['first_session_game_type'] = first_sessions['game_type'].values
first_session_features['first_session_is_sports'] = (first_sessions['game_type'] == 'sports_betting').astype(int).values
first_session_features['first_session_device'] = first_sessions['device_type'].values
first_session_features['first_session_country'] = first_sessions['country'].values

In [12]:
# Payment and account features
first_session_features['first_session_payment_method'] = first_sessions['payment_method'].values
first_session_features['first_session_account_age'] = first_sessions['account_age_days'].values
first_session_features['first_session_vip_tier'] = first_sessions['vip_tier'].values
first_session_features['first_session_user_age'] = first_sessions['user_age'].values

In [13]:
# Campaign and bonus features
first_session_features['first_session_campaign'] = first_sessions['campaign_type'].values
first_session_features['first_session_bonus_used'] = first_sessions['bonus_used'].values

In [14]:
# Betting behavior features
first_session_features['first_session_bet_amount'] = first_sessions['bet_amount'].values
first_session_features['first_session_win_amount'] = first_sessions['win_amount'].values
first_session_features['first_session_net_result'] = first_sessions['net_result'].values
first_session_features['first_session_won'] = (first_sessions['net_result'] > 0).astype(int).values

In [15]:
# Session behavior features
first_session_features['first_session_length'] = first_sessions['session_length_minutes'].values
first_session_features['first_session_games_played'] = first_sessions['games_played'].values

In [16]:
# Deposit/Withdrawal features
first_session_features['first_session_deposited'] = (first_sessions['deposit_amount'].fillna(0) > 0).astype(int).values
first_session_features['first_session_deposit_amount'] = first_sessions['deposit_amount'].fillna(0).values
first_session_features['first_session_withdrew'] = (first_sessions['withdrawal_amount'].fillna(0) > 0).astype(int).values

In [17]:
print(f"First Session Features: {first_session_features.shape}")
first_session_features.head()

First Session Features: (1446, 24)


,user_id,first_session_hour,first_session_day_of_week,first_session_weekend,first_session_holiday,first_session_game_type,first_session_is_sports,first_session_device,first_session_country,first_session_payment_method,first_session_account_age,first_session_vip_tier,first_session_user_age,first_session_campaign,first_session_bonus_used,first_session_bet_amount,first_session_win_amount,first_session_net_result,first_session_won,first_session_length,first_session_games_played,first_session_deposited,first_session_deposit_amount,first_session_withdrew
0,user_000002,16,1,0,1,blackjack,0,mobile,MX,e_wallet,17,gold,43.0,welcome_bonus,0,6731.21,12678.31,5947.10,1,144,48,1,3802.83,0
7,user_000003,19,3,0,0,blackjack,0,tablet,BR,e_wallet,3,gold,52.0,welcome_bonus,1,11278.64,0.00,-11278.64,0,240,80,1,5584.40,0
19,user_000004,22,4,0,0,poker,0,mobile,FR,e_wallet,6,gold,63.0,none,0,279.05,465.14,186.09,1,222,74,1,2565.21,0
30,user_000005,20,6,1,0,poker,0,tablet,MX,credit_card,29,bronze,42.0,free_spins,0,15451.79,0.00,-15451.79,0,376,125,0,0.00,0
38,user_000006,3,3,0,0,live_dealer,0,desktop,MX,bank_transfer,13,silver,42.0,reload_bonus,0,387.12,660.91,273.79,1,311,103,1,1852.56,1


---
## 4. Behavioral Aggregation Features

Extract aggregated behavioral features per user.

In [18]:
# Session count and timing aggregations
session_agg = df_processed.groupby('user_id').agg({
    'session_id': 'count',
    'timestamp': ['min', 'max']
})
session_agg.columns = ['total_sessions', 'first_session_time', 'last_session_time']

In [19]:
# Session characteristics aggregations
session_chars = df_processed.groupby('user_id').agg({
    'session_length_minutes': ['sum', 'mean', 'std', 'max', 'min'],
    'games_played': ['sum', 'mean', 'std', 'max']
})
session_chars.columns = [
    'total_time_played', 'avg_session_length', 'std_session_length', 
    'max_session_length', 'min_session_length',
    'total_games_played', 'avg_games_per_session', 'std_games_per_session', 'max_games_per_session'
]

In [20]:
# Session gap aggregations
gap_agg = df_processed.groupby('user_id')['previous_session_gap_hours'].agg(['mean', 'std', 'max', 'min'])
gap_agg.columns = ['avg_session_gap_hours', 'std_session_gap_hours', 'max_session_gap_hours', 'min_session_gap_hours']

In [21]:
# Temporal pattern aggregations
def get_mode(x):
    return x.mode()[0] if len(x.mode()) > 0 else x.median()

temporal_agg = df_processed.groupby('user_id').agg({
    'hour': get_mode,
    'day_of_week': get_mode,
    'is_weekend': 'mean',
    'is_holiday': 'mean'
})
temporal_agg.columns = ['preferred_hour', 'preferred_day', 'weekend_ratio', 'holiday_ratio']

In [22]:
# Diversity and bonus aggregations
diversity_agg = df_processed.groupby('user_id').agg({
    'game_type': 'nunique',
    'device_type': 'nunique',
    'bonus_used': ['sum', 'mean']
})
diversity_agg.columns = ['game_type_diversity', 'device_diversity', 'total_bonuses_used', 'bonus_usage_rate']

In [23]:
# Merge all behavioral features
behavioral_features = session_agg.join(session_chars).join(gap_agg).join(temporal_agg).join(diversity_agg)

In [24]:
# Calculate derived features
behavioral_features['active_days'] = (behavioral_features['last_session_time'] - behavioral_features['first_session_time']).dt.days + 1
behavioral_features['sessions_per_day'] = behavioral_features['total_sessions'] / behavioral_features['active_days'].replace(0, 1)

# Drop timestamp columns
behavioral_features = behavioral_features.drop(['first_session_time', 'last_session_time'], axis=1)
behavioral_features = behavioral_features.reset_index()

In [25]:
print(f"Behavioral Features: {behavioral_features.shape}")
behavioral_features.head()

Behavioral Features: (1446, 25)


,user_id,total_sessions,total_time_played,avg_session_length,std_session_length,max_session_length,min_session_length,total_games_played,avg_games_per_session,std_games_per_session,max_games_per_session,avg_session_gap_hours,std_session_gap_hours,max_session_gap_hours,min_session_gap_hours,preferred_hour,preferred_day,weekend_ratio,holiday_ratio,game_type_diversity,device_diversity,total_bonuses_used,bonus_usage_rate,active_days,sessions_per_day
0,user_000002,7,1289,184.142857,86.379837,373,121,427,61.000000,28.792360,124,-0.166667,0.408248,0.0,-1.0,0,3,0.000000,0.285714,4,3,2,0.285714,3,2.333333
1,user_000003,12,2193,182.750000,83.126546,284,17,726,60.500000,27.655346,94,0.000000,0.000000,0.0,0.0,11,6,0.333333,0.250000,3,2,4,0.333333,18,0.666667
2,user_000004,11,1587,144.272727,78.860752,298,41,525,47.727273,26.348020,99,0.000000,0.000000,0.0,0.0,3,4,0.272727,0.000000,3,3,2,0.181818,12,0.916667
3,user_000005,8,2005,250.625000,170.311427,576,62,666,83.250000,56.965528,192,0.000000,0.000000,0.0,0.0,15,3,0.125000,0.000000,3,3,0,0.000000,6,1.333333
4,user_000006,6,882,147.000000,98.608316,311,53,292,48.666667,32.794308,103,-0.200000,0.447214,0.0,-1.0,3,6,0.666667,0.166667,3,3,2,0.333333,4,1.500000


---
## 5. Financial Features

Extract financial and betting features per user.

In [26]:
# Betting aggregations
betting_agg = df_processed.groupby('user_id').agg({
    'bet_amount': ['sum', 'mean', 'std', 'max', 'min'],
    'win_amount': ['sum', 'mean', 'max'],
    'net_result': ['sum', 'mean', 'std', 'max', 'min']
})
betting_agg.columns = [
    'total_bet_amount', 'avg_bet_amount', 'std_bet_amount', 'max_bet_amount', 'min_bet_amount',
    'total_win_amount', 'avg_win_amount', 'max_win_amount',
    'total_net_result', 'avg_net_result', 'std_net_result', 'max_net_result', 'min_net_result'
]

In [27]:
# Deposit aggregations
deposit_agg = df_processed.groupby('user_id')['deposit_amount'].agg([
    ('total_deposits', lambda x: x.fillna(0).sum()),
    ('deposit_count', lambda x: (x.fillna(0) > 0).sum()),
    ('avg_deposit_amount', lambda x: x.fillna(0).mean())
])

In [28]:
# Withdrawal aggregations
withdrawal_agg = df_processed.groupby('user_id')['withdrawal_amount'].agg([
    ('total_withdrawals', lambda x: x.fillna(0).sum()),
    ('withdrawal_count', lambda x: (x.fillna(0) > 0).sum())
])

In [29]:
# Lifetime metrics
lifetime_agg = df_processed.groupby('user_id').agg({
    'lifetime_deposits': 'last',
    'lifetime_bets': 'last',
    'avg_bet_size': 'last'
})
lifetime_agg.columns = ['lifetime_deposits', 'lifetime_bets', 'lifetime_avg_bet_size']

In [30]:
# Merge all financial aggregations
financial_features = betting_agg.join(deposit_agg).join(withdrawal_agg).join(lifetime_agg)

In [31]:
# Calculate derived financial features
financial_features['net_deposits'] = financial_features['total_deposits'] - financial_features['total_withdrawals']
financial_features['win_rate'] = financial_features['total_win_amount'] / financial_features['total_bet_amount'].replace(0, 1)
financial_features['return_on_bets'] = financial_features['total_net_result'] / financial_features['total_bet_amount'].replace(0, 1)
financial_features['deposit_to_bet_ratio'] = financial_features['total_deposits'] / financial_features['total_bet_amount'].replace(0, 1)

In [32]:
# Calculate winning session rate
win_sessions = df_processed.groupby('user_id')['net_result'].apply(lambda x: (x > 0).sum())
total_sessions = df_processed.groupby('user_id').size()
financial_features['winning_session_rate'] = win_sessions / total_sessions
financial_features = financial_features.reset_index()

In [33]:
print(f"Financial Features: {financial_features.shape}")
financial_features.head()

Financial Features: (1446, 27)


,user_id,total_bet_amount,avg_bet_amount,std_bet_amount,max_bet_amount,min_bet_amount,total_win_amount,avg_win_amount,max_win_amount,total_net_result,avg_net_result,std_net_result,max_net_result,min_net_result,total_deposits,deposit_count,avg_deposit_amount,total_withdrawals,withdrawal_count,lifetime_deposits,lifetime_bets,lifetime_avg_bet_size,net_deposits,win_rate,return_on_bets,deposit_to_bet_ratio,winning_session_rate
0,user_000002,67308.72,9615.531429,17594.989647,49259.73,1112.86,15032.59,2147.512857,12678.31,-52276.13,-7468.018571,18729.092537,5947.10,-49259.73,28103.51,6,4014.787143,0.00,0,28103.51,366,10639.79,28103.51,0.223338,-0.776662,0.417531,0.285714
1,user_000003,1042709.84,86892.486667,179305.961521,620372.81,378.02,110889.40,9240.783333,72023.28,-931820.44,-77651.703333,184090.216073,43077.75,-620372.81,83210.82,9,6934.235000,3376.96,1,83210.82,726,86892.49,79833.86,0.106347,-0.893653,0.079802,0.333333
2,user_000004,177618.30,16147.118182,22702.686437,74159.19,279.05,215790.06,19617.278182,177395.74,38171.76,3470.160000,36364.346457,103236.55,-38329.60,64918.43,10,5901.675455,4548.40,1,64918.43,525,16147.12,60370.03,1.214909,0.214909,0.365494,0.363636
3,user_000005,36602.77,4575.346250,5173.257801,15451.79,268.86,6438.79,804.848750,4619.09,-30163.98,-3770.497500,5936.924331,2809.70,-15451.79,53180.11,7,6647.513750,0.00,0,53180.11,666,4971.68,53180.11,0.175910,-0.824090,1.452899,0.250000
4,user_000006,276841.90,46140.316667,100417.651560,250608.58,387.12,34721.20,5786.866667,28690.53,-242120.70,-40353.450000,103103.089994,9600.49,-250608.58,52883.48,5,8813.913333,2606.52,1,52883.48,292,46140.32,50276.96,0.125419,-0.874581,0.191024,0.666667


---
## 6. Engagement Features

Extract engagement and preference features per user.

In [34]:
# Initialize engagement features
engagement_features = pd.DataFrame()
engagement_features['user_id'] = df_processed['user_id'].unique()

In [35]:
# Favorite game type
favorite_game = df_processed.groupby('user_id')['game_type'].agg(
    lambda x: x.mode()[0] if len(x.mode()) > 0 else 'unknown'
).reset_index()
favorite_game.columns = ['user_id', 'favorite_game_type']
engagement_features = engagement_features.merge(favorite_game, on='user_id', how='left')

In [36]:
# Favorite device
favorite_device = df_processed.groupby('user_id')['device_type'].agg(
    lambda x: x.mode()[0] if len(x.mode()) > 0 else 'unknown'
).reset_index()
favorite_device.columns = ['user_id', 'primary_device']
engagement_features = engagement_features.merge(favorite_device, on='user_id', how='left')

In [37]:
# Favorite payment method
favorite_payment = df_processed.groupby('user_id')['payment_method'].agg(
    lambda x: x.mode()[0] if len(x.mode()) > 0 and x.notna().any() else 'unknown'
).reset_index()
favorite_payment.columns = ['user_id', 'primary_payment_method']
engagement_features = engagement_features.merge(favorite_payment, on='user_id', how='left')

In [38]:
# Mobile ratio
mobile_ratio = df_processed.groupby('user_id')['device_type'].apply(
    lambda x: (x == 'mobile').mean()
).reset_index()
mobile_ratio.columns = ['user_id', 'mobile_ratio']
engagement_features = engagement_features.merge(mobile_ratio, on='user_id', how='left')

In [39]:
# Night owl ratio (sessions between 10 PM and 6 AM)
night_ratio = df_processed.groupby('user_id')['hour'].apply(
    lambda x: ((x >= 22) | (x <= 6)).mean()
).reset_index()
night_ratio.columns = ['user_id', 'night_owl_ratio']
engagement_features = engagement_features.merge(night_ratio, on='user_id', how='left')

In [40]:
# Sports betting ratio
sports_ratio = df_processed.groupby('user_id')['game_type'].apply(
    lambda x: (x == 'sports_betting').mean()
).reset_index()
sports_ratio.columns = ['user_id', 'sports_betting_ratio']
engagement_features = engagement_features.merge(sports_ratio, on='user_id', how='left')

In [41]:
# Uses crypto
uses_crypto = df_processed.groupby('user_id')['payment_method'].apply(
    lambda x: (x == 'crypto').any()
).astype(int).reset_index()
uses_crypto.columns = ['user_id', 'uses_crypto']
engagement_features = engagement_features.merge(uses_crypto, on='user_id', how='left')

In [42]:
# Payment method diversity
payment_diversity = df_processed.groupby('user_id')['payment_method'].nunique().reset_index()
payment_diversity.columns = ['user_id', 'payment_method_diversity']
engagement_features = engagement_features.merge(payment_diversity, on='user_id', how='left')

In [43]:
# Campaign exposure diversity
campaign_diversity = df_processed.groupby('user_id')['campaign_type'].nunique().reset_index()
campaign_diversity.columns = ['user_id', 'campaign_diversity']
engagement_features = engagement_features.merge(campaign_diversity, on='user_id', how='left')

In [44]:
print(f"Engagement Features: {engagement_features.shape}")
engagement_features.head()

Engagement Features: (1446, 10)


,user_id,favorite_game_type,primary_device,primary_payment_method,mobile_ratio,night_owl_ratio,sports_betting_ratio,uses_crypto,payment_method_diversity,campaign_diversity
0,user_000002,blackjack,mobile,e_wallet,0.428571,0.285714,0.285714,0,1,4
1,user_000003,live_dealer,tablet,e_wallet,0.000000,0.583333,0.250000,0,2,4
2,user_000004,poker,mobile,e_wallet,0.363636,0.636364,0.272727,0,2,4
3,user_000005,sports_betting,mobile,credit_card,0.500000,0.375000,0.500000,0,2,5
4,user_000006,live_dealer,tablet,bank_transfer,0.166667,0.500000,0.333333,0,2,4


---
## 7. Trend Features (Behavioral Change Indicators)

Extract trend features that capture behavioral changes over time. These are strong indicators of churn/retention.

In [45]:
def calculate_trend(series):
    """Calculate linear trend slope"""
    if len(series) < 2:
        return 0
    x = np.arange(len(series))
    try:
        slope, _, _, _, _ = stats.linregress(x, series)
        return slope
    except:
        return 0

In [46]:
def calculate_first_vs_last_half(series):
    """Compare first half vs second half average"""
    if len(series) < 2:
        return 1
    mid = len(series) // 2
    first_half_avg = series[:mid].mean()
    second_half_avg = series[mid:].mean()
    if first_half_avg == 0:
        return 1
    return second_half_avg / first_half_avg

In [47]:
# Initialize trend features
trend_features = pd.DataFrame()
trend_features['user_id'] = df_processed['user_id'].unique()

# Filter users with multiple sessions
multi_session_users = df_processed.groupby('user_id').filter(lambda x: len(x) >= 3)

In [48]:
# Calculate trend features for multi-session users
if len(multi_session_users) > 0:
    # Bet amount trend
    bet_trend = multi_session_users.groupby('user_id')['bet_amount'].apply(calculate_trend).reset_index()
    bet_trend.columns = ['user_id', 'bet_amount_trend']
    trend_features = trend_features.merge(bet_trend, on='user_id', how='left')
    
    # Session length trend
    session_trend = multi_session_users.groupby('user_id')['session_length_minutes'].apply(calculate_trend).reset_index()
    session_trend.columns = ['user_id', 'session_length_trend']
    trend_features = trend_features.merge(session_trend, on='user_id', how='left')

In [49]:
# Additional trend features
if len(multi_session_users) > 0:
    # Games played trend
    games_trend = multi_session_users.groupby('user_id')['games_played'].apply(calculate_trend).reset_index()
    games_trend.columns = ['user_id', 'games_played_trend']
    trend_features = trend_features.merge(games_trend, on='user_id', how='left')
    
    # Bet amount decay
    bet_decay = multi_session_users.groupby('user_id')['bet_amount'].apply(calculate_first_vs_last_half).reset_index()
    bet_decay.columns = ['user_id', 'bet_amount_decay']
    trend_features = trend_features.merge(bet_decay, on='user_id', how='left')

In [50]:
# Session length decay
if len(multi_session_users) > 0:
    session_decay = multi_session_users.groupby('user_id')['session_length_minutes'].apply(calculate_first_vs_last_half).reset_index()
    session_decay.columns = ['user_id', 'session_length_decay']
    trend_features = trend_features.merge(session_decay, on='user_id', how='left')

In [51]:
# Fill NaN with neutral values
trend_cols = ['bet_amount_trend', 'session_length_trend', 'games_played_trend',
              'bet_amount_decay', 'session_length_decay']
for col in trend_cols:
    if col in trend_features.columns:
        trend_features[col] = trend_features[col].fillna(0)
    else:
        trend_features[col] = 0

In [52]:
# Session gap trend (increasing gaps = disengagement signal)
gap_df = df_processed[df_processed['previous_session_gap_hours'].notna()]
if len(gap_df) > 0:
    gap_trend = gap_df.groupby('user_id')['previous_session_gap_hours'].apply(calculate_trend).reset_index()
    gap_trend.columns = ['user_id', 'session_gap_trend']
    trend_features = trend_features.merge(gap_trend, on='user_id', how='left')
    trend_features['session_gap_trend'] = trend_features['session_gap_trend'].fillna(0)
else:
    trend_features['session_gap_trend'] = 0

In [53]:
print(f"Trend Features: {trend_features.shape}")
trend_features.head()

Trend Features: (1446, 7)


,user_id,bet_amount_trend,session_length_trend,games_played_trend,bet_amount_decay,session_length_decay,session_gap_trend
0,user_000002,1561.310357,-6.750000,-2.285714,4.632103,0.708145,-0.085714
1,user_000003,13367.342517,3.744755,1.237762,7.888904,1.296335,0.000000
2,user_000004,187.712636,6.345455,2.072727,0.844843,1.303177,0.000000
3,user_000005,-487.232500,-4.059524,-1.357143,0.970429,1.342290,0.000000
4,user_000006,21505.253143,-35.085714,-11.600000,12.756741,0.676806,0.100000


---
## 8. Merge All Features

In [54]:
# Start with first session features
features_df = first_session_features.copy()

# Merge behavioral features
features_df = features_df.merge(behavioral_features, on='user_id', how='left')

print(f"After behavioral merge: {features_df.shape}")

After behavioral merge: (1446, 48)


In [55]:
# Merge financial features
features_df = features_df.merge(financial_features, on='user_id', how='left')

print(f"After financial merge: {features_df.shape}")

After financial merge: (1446, 74)


In [56]:
# Merge engagement features
features_df = features_df.merge(engagement_features, on='user_id', how='left')

print(f"After engagement merge: {features_df.shape}")

After engagement merge: (1446, 83)


In [57]:
# Merge trend features
features_df = features_df.merge(trend_features, on='user_id', how='left')

print(f"After trend merge: {features_df.shape}")

After trend merge: (1446, 89)


In [ ]:
# Add target (churn)
features_df = features_df.merge(target.reset_index(), on='user_id', how='left')

print(f"\nCombined Features Dataset: {features_df.shape}")
print(f"Target column: 'churn' (1=churned, 0=retained)")

In [59]:
print(f"Feature Categories:")
print(f"- First Session Features: ~25")
print(f"- Behavioral Features: ~20")
print(f"- Financial Features: ~20")
print(f"- Engagement Features: ~10")
print(f"- Trend Features: ~6")

Feature Categories:
- First Session Features: ~25
- Behavioral Features: ~20
- Financial Features: ~20
- Engagement Features: ~10
- Trend Features: ~6


In [60]:
# Preview combined features
features_df.head()

,user_id,first_session_hour,first_session_day_of_week,first_session_weekend,first_session_holiday,first_session_game_type,first_session_is_sports,first_session_device,first_session_country,first_session_payment_method,first_session_account_age,first_session_vip_tier,first_session_user_age,first_session_campaign,first_session_bonus_used,first_session_bet_amount,first_session_win_amount,first_session_net_result,first_session_won,first_session_length,first_session_games_played,first_session_deposited,first_session_deposit_amount,first_session_withdrew,total_sessions,...,withdrawal_count,lifetime_deposits,lifetime_bets,lifetime_avg_bet_size,net_deposits,win_rate,return_on_bets,deposit_to_bet_ratio,winning_session_rate,favorite_game_type,primary_device,primary_payment_method,mobile_ratio,night_owl_ratio,sports_betting_ratio,uses_crypto,payment_method_diversity,campaign_diversity,bet_amount_trend,session_length_trend,games_played_trend,bet_amount_decay,session_length_decay,session_gap_trend,target
0,user_000002,16,1,0,1,blackjack,0,mobile,MX,e_wallet,17,gold,43.0,welcome_bonus,0,6731.21,12678.31,5947.10,1,144,48,1,3802.83,0,7,...,0,28103.51,366,10639.79,28103.51,0.223338,-0.776662,0.417531,0.285714,blackjack,mobile,e_wallet,0.428571,0.285714,0.285714,0,1,4,1561.310357,-6.750000,-2.285714,4.632103,0.708145,-0.085714,1
1,user_000003,19,3,0,0,blackjack,0,tablet,BR,e_wallet,3,gold,52.0,welcome_bonus,1,11278.64,0.00,-11278.64,0,240,80,1,5584.40,0,12,...,1,83210.82,726,86892.49,79833.86,0.106347,-0.893653,0.079802,0.333333,live_dealer,tablet,e_wallet,0.000000,0.583333,0.250000,0,2,4,13367.342517,3.744755,1.237762,7.888904,1.296335,0.000000,1
2,user_000004,22,4,0,0,poker,0,mobile,FR,e_wallet,6,gold,63.0,none,0,279.05,465.14,186.09,1,222,74,1,2565.21,0,11,...,1,64918.43,525,16147.12,60370.03,1.214909,0.214909,0.365494,0.363636,poker,mobile,e_wallet,0.363636,0.636364,0.272727,0,2,4,187.712636,6.345455,2.072727,0.844843,1.303177,0.000000,1
3,user_000005,20,6,1,0,poker,0,tablet,MX,credit_card,29,bronze,42.0,free_spins,0,15451.79,0.00,-15451.79,0,376,125,0,0.00,0,8,...,0,53180.11,666,4971.68,53180.11,0.175910,-0.824090,1.452899,0.250000,sports_betting,mobile,credit_card,0.500000,0.375000,0.500000,0,2,5,-487.232500,-4.059524,-1.357143,0.970429,1.342290,0.000000,1
4,user_000006,3,3,0,0,live_dealer,0,desktop,MX,bank_transfer,13,silver,42.0,reload_bonus,0,387.12,660.91,273.79,1,311,103,1,1852.56,1,6,...,1,52883.48,292,46140.32,50276.96,0.125419,-0.874581,0.191024,0.666667,live_dealer,tablet,bank_transfer,0.166667,0.500000,0.333333,0,2,4,21505.253143,-35.085714,-11.600000,12.756741,0.676806,0.100000,1


In [61]:
# Check feature types
print("Feature Data Types:")
print(features_df.dtypes.value_counts())

Feature Data Types:
float64    49
int64      31
object     10
Name: count, dtype: int64


---
## 9. Handle Missing Values

In [62]:
# Check missing values
missing = features_df.isnull().sum()
missing_pct = (missing / len(features_df) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing %', ascending=False)

print("Columns with Missing Values:")
missing_df

Columns with Missing Values:


,Missing Count,Missing %
first_session_user_age,116,8.02
std_session_gap_hours,79,5.46
first_session_vip_tier,45,3.11
first_session_payment_method,41,2.84
first_session_game_type,35,2.42
std_session_length,27,1.87
std_games_per_session,27,1.87
avg_session_gap_hours,27,1.87
max_session_gap_hours,27,1.87
min_session_gap_hours,27,1.87


In [63]:
def handle_missing_values(df):
    """
    Handle missing values based on feature type and business logic.
    """
    df = df.copy()
    
    # Numerical features - fill with median
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    for col in numerical_cols:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].median())
    
    # Categorical features - fill with 'unknown'
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if df[col].isnull().any():
            df[col] = df[col].fillna('unknown')
    
    return df

In [64]:
# Apply missing value handling
features_clean = handle_missing_values(features_df)

# Verify no missing values
print(f"Remaining missing values: {features_clean.isnull().sum().sum()}")

Remaining missing values: 0


---
## 10. Encode Categorical Variables

In [65]:
# Identify categorical columns
categorical_cols = features_clean.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('user_id')  # Keep user_id as is

print(f"Categorical columns to encode: {len(categorical_cols)}")
for col in categorical_cols:
    print(f"  - {col}: {features_clean[col].nunique()} unique values")

Categorical columns to encode: 9
  - first_session_game_type: 7 unique values
  - first_session_device: 3 unique values
  - first_session_country: 11 unique values
  - first_session_payment_method: 6 unique values
  - first_session_vip_tier: 5 unique values
  - first_session_campaign: 5 unique values
  - favorite_game_type: 7 unique values
  - primary_device: 3 unique values
  - primary_payment_method: 6 unique values


In [66]:
# VIP tier - ordinal encoding
vip_order = {'unknown': 0, 'bronze': 1, 'silver': 2, 'gold': 3, 'platinum': 4, 'diamond': 5}
features_encoded = features_clean.copy()

if 'first_session_vip_tier' in features_encoded.columns:
    features_encoded['first_session_vip_tier_encoded'] = features_encoded['first_session_vip_tier'].map(vip_order).fillna(0)

In [67]:
# One-hot encode low cardinality features
low_cardinality_cols = [
    'first_session_device', 'first_session_game_type', 'first_session_campaign',
    'primary_device', 'favorite_game_type'
]

for col in low_cardinality_cols:
    if col in features_encoded.columns:
        dummies = pd.get_dummies(features_encoded[col], prefix=col, drop_first=True)
        features_encoded = pd.concat([features_encoded, dummies], axis=1)

In [ ]:
# Target encode high cardinality features
high_cardinality_cols = [
    'first_session_country', 'first_session_payment_method',
    'primary_payment_method'
]

for col in high_cardinality_cols:
    if col in features_encoded.columns:
        target_means = features_encoded.groupby(col)['churn'].mean()
        features_encoded[f'{col}_encoded'] = features_encoded[col].map(target_means).fillna(features_encoded['churn'].mean())

In [69]:
# Drop original categorical columns
cols_to_drop = [col for col in categorical_cols if col in features_encoded.columns]
features_encoded = features_encoded.drop(columns=cols_to_drop)

print(f"Encoded Features Dataset: {features_encoded.shape}")

Encoded Features Dataset: (1446, 105)


In [70]:
def add_cyclical_features(df):
    """Add sin/cos encoding for cyclical features"""
    df = df.copy()
    
    # Hour encoding
    if 'first_session_hour' in df.columns:
        df['first_session_hour_sin'] = np.sin(2 * np.pi * df['first_session_hour'] / 24)
        df['first_session_hour_cos'] = np.cos(2 * np.pi * df['first_session_hour'] / 24)
    
    if 'preferred_hour' in df.columns:
        df['preferred_hour_sin'] = np.sin(2 * np.pi * df['preferred_hour'] / 24)
        df['preferred_hour_cos'] = np.cos(2 * np.pi * df['preferred_hour'] / 24)
    
    return df

In [71]:
def add_day_cyclical_features(df):
    """Add sin/cos encoding for day of week"""
    df = df.copy()
    
    if 'first_session_day_of_week' in df.columns:
        df['first_session_dow_sin'] = np.sin(2 * np.pi * df['first_session_day_of_week'] / 7)
        df['first_session_dow_cos'] = np.cos(2 * np.pi * df['first_session_day_of_week'] / 7)
    
    if 'preferred_day' in df.columns:
        df['preferred_day_sin'] = np.sin(2 * np.pi * df['preferred_day'] / 7)
        df['preferred_day_cos'] = np.cos(2 * np.pi * df['preferred_day'] / 7)
    
    return df

In [72]:
# Apply cyclical encodings
features_final = add_cyclical_features(features_encoded)
features_final = add_day_cyclical_features(features_final)

print(f"Final Features Dataset: {features_final.shape}")

Final Features Dataset: (1446, 113)


---
## 11. Feature Analysis

In [ ]:
# Get numeric columns for correlation
numeric_cols = features_final.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['user_id', 'churn']]

In [ ]:
# Correlation with target (churn)
target_correlations = features_final[numeric_cols + ['churn']].corr()['churn'].drop('churn')
target_correlations = target_correlations.sort_values(key=abs, ascending=False)

print("Top 20 Features Correlated with Churn Target:")
print("="*50)
print("(Positive = higher value leads to MORE churn)")
print("(Negative = higher value leads to LESS churn)")
print("="*50)
for feature, corr in target_correlations.head(20).items():
    print(f"{feature:45} {corr:+.4f}")

In [ ]:
# Visualize top correlations with churn
fig, ax = plt.subplots(figsize=(10, 10))

top_features = target_correlations.head(20)
# For churn: negative correlation = reduces churn (green/good), positive = increases churn (red/bad)
colors = ['#e74c3c' if x > 0 else '#27ae60' for x in top_features.values]

ax.barh(range(len(top_features)), top_features.values, color=colors)
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features.index)
ax.set_xlabel('Correlation with Churn Target')
ax.set_title('Top 20 Features Correlated with Churn\n(Red = increases churn, Green = reduces churn)', 
             fontsize=14, fontweight='bold')
ax.axvline(0, color='black', linewidth=0.5)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

---
## 12. Save Final Dataset

In [76]:
# Save the final feature dataset
features_final.to_csv('../data/features_engineered.csv', index=False)
print(f"Saved engineered features: {features_final.shape}")

Saved engineered features: (1446, 113)


In [ ]:
# Save feature names for reference
feature_names = [col for col in features_final.columns if col not in ['user_id', 'churn']]
with open('../data/feature_names.txt', 'w') as f:
    for name in feature_names:
        f.write(f"{name}\n")

print(f"Total features: {len(feature_names)}")

In [ ]:
# Summary statistics
print("\nFeature Engineering Summary")
print("="*50)
print(f"Total Users: {len(features_final):,}")
print(f"Total Features: {len(feature_names)}")
print(f"Churn Rate: {features_final['churn'].mean():.2%}")
print(f"Retention Rate: {1 - features_final['churn'].mean():.2%}")
print(f"\nFeature Categories:")
print(f"  - Numerical: {features_final.select_dtypes(include=[np.number]).shape[1]}")
print(f"  - Boolean/Binary: {sum(features_final[feature_names].nunique() == 2)}")

---
## Next Steps

The next notebook (`03_modeling_and_evaluation.ipynb`) will:
1. Split data into train/validation/test sets
2. Train multiple models (Logistic Regression, Random Forest, XGBoost, LightGBM)
3. Perform hyperparameter tuning
4. Evaluate models with ML and business metrics
5. Analyze feature importance and SHAP values